# Компʼютерний практикум №11
Виконав студент групи ЗК-41мп Гломозда Костянтин

СПОСОБИ ПОБУДОВИ МОДЕЛІ У PYTORCH

Download data

In [1]:
import torchvision
import torch.nn as nn
import torch
import numpy as np
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

def extract_data(loader):
    X, y = [], []
    for data in loader:
        inputs, labels = data
        X.append(inputs)
        y.append(labels)
    return torch.cat(X), torch.cat(y)

X_train, y_train = extract_data(train_loader)
X_val, y_val = extract_data(val_loader)
X_test, y_test = extract_data(test_loader)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, {y_val.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

Files already downloaded and verified
Files already downloaded and verified
Training data shape: torch.Size([40000, 3, 32, 32]), torch.Size([40000])
Validation data shape: torch.Size([10000, 3, 32, 32]), torch.Size([10000])
Testing data shape: torch.Size([10000, 3, 32, 32]), torch.Size([10000])


Train function

In [ ]:
def train_with_l2(net, X_train, y_train, X_test, y_test, epochs=30, batch_size=100, weight_decay=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=weight_decay)
    
    test_accuracy_history = []
    test_loss_history = []
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    
    for epoch in range(epochs):
        order = np.random.permutation(len(X_train))
        
        for start_index in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            net.train()
            batch_indexes = order[start_index:start_index+batch_size]
            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)
            preds = net.forward(X_batch)
            loss_value = loss(preds, y_batch)
            loss_value.backward()
            optimizer.step()
        
        net.eval()
        test_preds = net.forward(X_test)
        test_loss_history.append(loss(test_preds, y_test).data.cpu())
        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
        test_accuracy_history.append(accuracy)
        print(f"Epoch: {epoch}/{epochs}, Accuracy: {accuracy:.3f}, Loss: {test_loss_history[-1]:.3f}")
    return test_accuracy_history, test_loss_history

За основу використаємо останню створену модель з 10 роботи

In [2]:
import torch
import numpy as np


class ModifiedNet(torch.nn.Module):
    def __init__(self):
        super(ModifiedNet, self).__init__()
        
        # BatchNorm for input normalization (3 channels for RGB)
        self.bn_input = torch.nn.BatchNorm2d(3)

        # First conv block: Conv2d -> ReLU -> BatchNorm -> MaxPool
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(16)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Second conv block: Conv2d -> ReLU -> BatchNorm -> MaxPool
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Third conv block: Conv2d -> ReLU -> BatchNorm -> MaxPool
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(64)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = torch.nn.Linear(64 * 4 * 4, 256)
        self.bn_fc1 = torch.nn.BatchNorm1d(256)
        self.fc2 = torch.nn.Linear(256, 64)
        self.bn_fc2 = torch.nn.BatchNorm1d(64)
        self.fc3 = torch.nn.Linear(64, 10)

        # Activation functions
        self.relu = torch.nn.ReLU()
        self.tanh = torch.nn.Tanh()

        # Dropout layers
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.bn_input(x)
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.pool3(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.bn_fc1(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.tanh(x)
        x = self.bn_fc2(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

# Training the model with the modified architecture
net = ModifiedNet()
accuracy_history, loss_history = train_with_l2(net, X_train, y_train, X_test, y_test, epochs=10, weight_decay=1e-4)
print(f"Final accuracy: {accuracy_history[-1]:.3f}")
print(f"Final loss: {loss_history[-1]:.3f}")

Epoch: 0/10, Accuracy: 0.597, Loss: 1.143
Epoch: 1/10, Accuracy: 0.657, Loss: 0.983
Epoch: 2/10, Accuracy: 0.683, Loss: 0.911
Epoch: 3/10, Accuracy: 0.692, Loss: 0.883
Epoch: 4/10, Accuracy: 0.711, Loss: 0.834
Epoch: 5/10, Accuracy: 0.721, Loss: 0.807
Epoch: 6/10, Accuracy: 0.733, Loss: 0.783
Epoch: 7/10, Accuracy: 0.725, Loss: 0.807
Epoch: 8/10, Accuracy: 0.730, Loss: 0.798
Epoch: 9/10, Accuracy: 0.741, Loss: 0.769
Final accuracy: 0.741
Final loss: 0.769


Module

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 28 * 28, 1024)
        self.fc2 = nn.Linear(1024, n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

# Training the model with the modified architecture
net = ModifiedNet()
accuracy_history, loss_history = train_with_l2(net, X_train, y_train, X_test, y_test, epochs=10, weight_decay=1e-4)
print(f"Final accuracy: {accuracy_history[-1]:.3f}")
print(f"Final loss: {loss_history[-1]:.3f}")

Epoch: 0/10, Accuracy: 0.601, Loss: 1.123
Epoch: 1/10, Accuracy: 0.647, Loss: 1.022
Epoch: 2/10, Accuracy: 0.684, Loss: 0.914
Epoch: 3/10, Accuracy: 0.703, Loss: 0.874
Epoch: 4/10, Accuracy: 0.716, Loss: 0.838
Epoch: 5/10, Accuracy: 0.723, Loss: 0.817
Epoch: 6/10, Accuracy: 0.729, Loss: 0.800
Epoch: 7/10, Accuracy: 0.733, Loss: 0.803
Epoch: 8/10, Accuracy: 0.728, Loss: 0.817
Epoch: 9/10, Accuracy: 0.730, Loss: 0.807
Final accuracy: 0.730
Final loss: 0.807


Sequential

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

class ModifiedNet(nn.Module):
    def __init__(self):
        super(ModifiedNet, self).__init__()
        self.encoder = nn.Sequential(
            conv_block(3, 32, kernel_size=3, padding=1),
            conv_block(32, 64, kernel_size=3, padding=1)
        )
        self.decoder = nn.Sequential(
            nn.Linear(64 * 32 * 32, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, 10)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

net = ModifiedNet()
accuracy_history, loss_history = train_with_l2(net, X_train, y_train, X_test, y_test, epochs=10, weight_decay=1e-4)
print(f"Final accuracy: {accuracy_history[-1]:.3f}")
print(f"Final loss: {loss_history[-1]:.3f}")

Epoch: 0/10, Accuracy: 0.504, Loss: 1.371
Epoch: 1/10, Accuracy: 0.552, Loss: 1.265
Epoch: 2/10, Accuracy: 0.554, Loss: 1.221
Epoch: 3/10, Accuracy: 0.571, Loss: 1.160
Epoch: 4/10, Accuracy: 0.598, Loss: 1.103
Epoch: 5/10, Accuracy: 0.609, Loss: 1.094
Epoch: 6/10, Accuracy: 0.636, Loss: 1.029
Epoch: 7/10, Accuracy: 0.660, Loss: 0.970
Epoch: 8/10, Accuracy: 0.652, Loss: 0.983
Epoch: 9/10, Accuracy: 0.663, Loss: 0.960
Final accuracy: 0.663
Final loss: 0.960


Dynamic Sequential

In [5]:
import torch
import torch.nn as nn

def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

def dec_block(in_f, out_f):
    return nn.Sequential(
        nn.Linear(in_f, out_f),
        nn.Sigmoid()
    )

class MyEncoder(nn.Module):
    def __init__(self, enc_sizes):
        super().__init__()
        self.conv_blocks = nn.Sequential(*[conv_block(in_f, out_f, kernel_size=3, padding=1)
                                           for in_f, out_f in zip(enc_sizes, enc_sizes[1:])])

    def forward(self, x):
        return self.conv_blocks(x)

class MyDecoder(nn.Module):
    def __init__(self, dec_sizes, n_classes):
        super().__init__()
        self.dec_blocks = nn.Sequential(*[dec_block(in_f, out_f)
                                          for in_f, out_f in zip(dec_sizes, dec_sizes[1:])])
        self.last = nn.Linear(dec_sizes[-1], n_classes)

    def forward(self, x):
        x = self.dec_blocks(x)
        return self.last(x)

class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, enc_sizes, dec_sizes, n_classes):
        super().__init__()
        self.enc_sizes = [in_c, *enc_sizes]
        self.dec_sizes = [enc_sizes[-1] * 32 * 32, *dec_sizes]
        self.encoder = MyEncoder(self.enc_sizes)
        self.decoder = MyDecoder(self.dec_sizes, n_classes)

    def forward(self, x):
        x = self.encoder(x)
        x = x.flatten(1)
        x = self.decoder(x)
        return x

net = MyCNNClassifier(3, [32, 64], [1024, 512], 10)
accuracy_history, loss_history = train_with_l2(net, X_train, y_train, X_test, y_test, epochs=10, weight_decay=1e-4)
print(f"Final accuracy: {accuracy_history[-1]:.3f}")
print(f"Final loss: {loss_history[-1]:.3f}")

Epoch: 0/10, Accuracy: 0.435, Loss: 1.468
Epoch: 1/10, Accuracy: 0.482, Loss: 1.413
Epoch: 2/10, Accuracy: 0.488, Loss: 1.351
Epoch: 3/10, Accuracy: 0.507, Loss: 1.343
Epoch: 4/10, Accuracy: 0.506, Loss: 1.334
Epoch: 5/10, Accuracy: 0.544, Loss: 1.237
Epoch: 6/10, Accuracy: 0.552, Loss: 1.238
Epoch: 7/10, Accuracy: 0.589, Loss: 1.117
Epoch: 8/10, Accuracy: 0.591, Loss: 1.148
Epoch: 9/10, Accuracy: 0.630, Loss: 1.035
Final accuracy: 0.630
Final loss: 1.035


ModuleList

Better train for less vram consumption

In [2]:
def train_with_l2_optimized(net, X_train, y_train, X_test, y_test, epochs=10, batch_size=50, weight_decay=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=weight_decay)
    
    test_accuracy_history = []
    test_loss_history = []
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    
    for epoch in range(epochs):
        order = np.random.permutation(len(X_train))
        
        for start_index in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            net.train()
            
            batch_indexes = order[start_index:start_index+batch_size]
            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)
            
            preds = net.forward(X_batch)
            loss_value = loss(preds, y_batch)
            loss_value.backward()
            optimizer.step()
        
        with torch.no_grad():  # Disable gradient computation for validation/testing
            net.eval()
            test_preds = net.forward(X_test)
            test_loss_history.append(loss(test_preds, y_test).data.cpu())
            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
            test_accuracy_history.append(accuracy)
            print(f"Epoch: {epoch}/{epochs}, Accuracy: {accuracy:.3f}, Loss: {test_loss_history[-1]:.3f}")
        
        # Clear trace after each epoch to free memory
        net.trace = []
    
    return test_accuracy_history, test_loss_history

In [5]:
import torch
import torch.nn as nn

def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, enc_sizes, dec_sizes, n_classes):
        super().__init__()
        self.encoder = nn.ModuleList([conv_block(in_f, out_f, kernel_size=3, padding=1)
                                      for in_f, out_f in zip([in_c] + enc_sizes, enc_sizes)])
        self.decoder = nn.ModuleList([nn.Linear(in_f, out_f)
                                      for in_f, out_f in zip([enc_sizes[-1] * 32 * 32] + dec_sizes, dec_sizes)])
        self.last = nn.Linear(dec_sizes[-1], n_classes)

    def forward(self, x):
        self.trace = []  # Clear trace at the start of the forward pass to avoid memory buildup
        for layer in self.encoder:
            x = layer(x)
            self.trace.append(x)
        x = x.view(x.size(0), -1)
        for layer in self.decoder:
            x = layer(x)
            self.trace.append(x)
        return self.last(x)

net = MyCNNClassifier(3, [32, 64], [1024, 512], 10)

accuracy_history, loss_history = train_with_l2_optimized(net, X_train, y_train, X_test, y_test, epochs=10, batch_size=50, weight_decay=1e-4)
print(f"Final accuracy: {accuracy_history[-1]:.3f}")
print(f"Final loss: {loss_history[-1]:.3f}")

Epoch: 0/10, Accuracy: 0.519, Loss: 1.431
Epoch: 1/10, Accuracy: 0.586, Loss: 1.163
Epoch: 2/10, Accuracy: 0.613, Loss: 1.134
Epoch: 3/10, Accuracy: 0.644, Loss: 1.049
Epoch: 4/10, Accuracy: 0.618, Loss: 1.168
Epoch: 5/10, Accuracy: 0.641, Loss: 1.127
Epoch: 6/10, Accuracy: 0.628, Loss: 1.267
Epoch: 7/10, Accuracy: 0.630, Loss: 1.408
Epoch: 8/10, Accuracy: 0.635, Loss: 1.547
Epoch: 9/10, Accuracy: 0.622, Loss: 1.734
Final accuracy: 0.622
Final loss: 1.734


ModuleDict

In [3]:
import torch
import torch.nn as nn

def conv_block(in_f, out_f, activation='relu', *args, **kwargs):
    activations = nn.ModuleDict([
        ['lrelu', nn.LeakyReLU()],
        ['relu', nn.ReLU()]
    ])
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        activations[activation]
    )

def dec_block(in_f, out_f):
    return nn.Sequential(
        nn.Linear(in_f, out_f),
        nn.Sigmoid()
    )

class MyEncoder(nn.Module):
    def __init__(self, enc_sizes, activation='relu', *args, **kwargs):
        super().__init__()
        self.conv_blocks = nn.Sequential(*[conv_block(in_f, out_f, activation=activation, kernel_size=3, padding=1, *args, **kwargs)
                                           for in_f, out_f in zip(enc_sizes, enc_sizes[1:])])

    def forward(self, x):
        return self.conv_blocks(x)

class MyDecoder(nn.Module):
    def __init__(self, dec_sizes, n_classes):
        super().__init__()
        self.dec_blocks = nn.Sequential(*[dec_block(in_f, out_f)
                                          for in_f, out_f in zip(dec_sizes, dec_sizes[1:])])
        self.last = nn.Linear(dec_sizes[-1], n_classes)

    def forward(self, x):
        return self.dec_blocks(x)

class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, enc_sizes, dec_sizes, n_classes, activation='relu'):
        super().__init__()
        self.enc_sizes = [in_c, *enc_sizes]
        self.dec_sizes = [enc_sizes[-1] * 32 * 32, *dec_sizes]
        self.encoder = MyEncoder(self.enc_sizes, activation=activation)
        self.decoder = MyDecoder(self.dec_sizes, n_classes)

    def forward(self, x):
        x = self.encoder(x)
        x = x.flatten(1)
        return self.decoder(x)

net = MyCNNClassifier(3, [32, 64], [1024, 512], 10, activation='lrelu')
accuracy_history, loss_history = train_with_l2_optimized(net, X_train, y_train, X_test, y_test, epochs=10, weight_decay=1e-4)
print(f"Final accuracy: {accuracy_history[-1]:.3f}")
print(f"Final loss: {loss_history[-1]:.3f}")

Epoch: 0/10, Accuracy: 0.100, Loss: 5.272
Epoch: 1/10, Accuracy: 0.100, Loss: 5.272
Epoch: 2/10, Accuracy: 0.086, Loss: 5.272
Epoch: 3/10, Accuracy: 0.100, Loss: 5.272
Epoch: 4/10, Accuracy: 0.100, Loss: 5.272
Epoch: 5/10, Accuracy: 0.100, Loss: 5.272
Epoch: 6/10, Accuracy: 0.100, Loss: 5.272
Epoch: 7/10, Accuracy: 0.100, Loss: 5.272
Epoch: 8/10, Accuracy: 0.100, Loss: 5.272
Epoch: 9/10, Accuracy: 0.100, Loss: 5.272
Final accuracy: 0.100
Final loss: 5.272


Результати:

Основа:
Final accuracy: 0.741
Final loss: 0.769

Module:
Final accuracy: 0.730
Final loss: 0.807

Sequential:
Final accuracy: 0.663
Final loss: 0.960

Dynamic Sequential:
Final accuracy: 0.630
Final loss: 1.035

ModuleList (оптимізовано для low vram):
Final accuracy: 0.622
Final loss: 1.734

ModuleDict (оптимізовано для low vram):
Final accuracy: 0.100
Final loss: 5.272

Висновок: Найкращі результати були отримані з використанням Module та Sequential